In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import ast

### Import first dataset

In [293]:
df_recipes2 = pd.read_parquet('../Data/recipes.parquet')
df_recipes2.head()

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
0,38.0,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09 21:46:00+00:00,Make and share this Low-Fat Berry Blue Frozen ...,[https://img.sndimg.com/food/image/upload/w_55...,...,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,None,"[Toss 2 cups berries with sugar., Let stand fo..."
1,39.0,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29 13:12:00+00:00,Make and share this Biryani recipe from Food.com.,[https://img.sndimg.com/food/image/upload/w_55...,...,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,None,[Soak saffron in warm milk for 5 minutes and p...
2,40.0,Best Lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,1999-09-05 19:52:00+00:00,This is from one of my first Good House Keepi...,[https://img.sndimg.com/food/image/upload/w_55...,...,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,None,"[Into a 1 quart Jar with tight fitting lid, pu..."
3,41.0,Carina's Tofu-Vegetable Kebabs,1586,Cyclopz,PT20M,PT24H,PT24H20M,1999-09-03 14:54:00+00:00,This dish is best prepared a day in advance to...,[https://img.sndimg.com/food/image/upload/w_55...,...,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,2.0,4 kebabs,"[Drain the tofu, carefully squeezing out exces..."
4,42.0,Cabbage Soup,1538,Duckie067,PT30M,PT20M,PT50M,1999-09-19 06:19:00+00:00,Make and share this Cabbage Soup recipe from F...,[https://img.sndimg.com/food/image/upload/w_55...,...,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0,None,"[Mix everything together and bring to a boil.,..."


In [294]:
# Drop unnecessary columns
columns_to_drop = ['RecipeId', 'AuthorId', 'DatePublished', 'RecipeYield', 'SaturatedFatContent', 'CholesterolContent', 'SodiumContent', 'CarbohydrateContent']
df_recipes2 = df_recipes2.drop(columns=columns_to_drop)

In [295]:
df_recipes2.isna().sum()

Name                               0
AuthorName                         0
CookTime                       82545
PrepTime                           0
TotalTime                          0
Description                        5
Images                             1
RecipeCategory                   751
Keywords                           0
RecipeIngredientQuantities         0
RecipeIngredientParts              0
AggregatedRating              253223
ReviewCount                   247489
Calories                           0
FatContent                         0
FiberContent                       0
SugarContent                       0
ProteinContent                     0
RecipeServings                182911
RecipeInstructions                 0
dtype: int64

#### Nan values

In [296]:
df_recipes2['CookTime'] = df_recipes2['CookTime'].fillna('PT0M')

In [302]:
df_recipes2 = df_recipes2.dropna().reset_index(drop=True)

In [303]:
df_recipes2.describe()

,AggregatedRating,ReviewCount,Calories,FatContent,FiberContent,SugarContent,ProteinContent,RecipeServings
count,172212.000000,172212.000000,172212.000000,172212.000000,172212.000000,172212.000000,172212.000000,172212.000000
mean,4.628310,5.155425,355.872155,17.854535,3.046740,12.368208,15.797985,8.552737
std,0.640551,20.970397,416.620332,28.599692,4.714552,34.610331,22.089223,138.494977
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,4.500000,1.000000,165.100000,5.300000,0.800000,2.200000,3.500000,4.000000
50%,5.000000,2.000000,285.600000,12.300000,2.000000,5.500000,8.800000,6.000000
75%,5.000000,4.000000,448.600000,23.100000,3.900000,14.200000,24.400000,8.000000
max,5.000000,3063.000000,41770.200000,4701.100000,835.700000,7565.200000,3270.300000,32767.000000


In [ ]:
# Filter outliers
df_recipes2 = df_recipes2[df_recipes2['RecipeServings']<=50]

In [305]:
df_recipes2

,Name,AuthorName,CookTime,PrepTime,TotalTime,Description,Images,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,AggregatedRating,ReviewCount,Calories,FatContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeInstructions
0,Low-Fat Berry Blue Frozen Dessert,Dancer,PT24H,PT45M,PT24H45M,Make and share this Low-Fat Berry Blue Frozen ...,[https://img.sndimg.com/food/image/upload/w_55...,Frozen Desserts,"[Dessert, Low Protein, Low Cholesterol, Health...","[4, 1⁄4, 1, 1]","[blueberries, granulated sugar, vanilla yogurt...",4.5,4.0,170.9,2.5,3.6,30.2,3.2,4.0,"[Toss 2 cups berries with sugar., Let stand fo..."
1,Biryani,elly9812,PT25M,PT4H,PT4H25M,Make and share this Biryani recipe from Food.com.,[https://img.sndimg.com/food/image/upload/w_55...,Chicken Breast,"[Chicken Thigh & Leg, Chicken, Poultry, Meat, ...","[1, 4, 2, 2, 8, 1⁄4, 8, 1⁄2, 1, 1, 1⁄4, 1⁄4, 1...","[saffron, milk, hot green chili peppers, onion...",3.0,1.0,1110.7,58.8,9.0,20.4,63.4,6.0,[Soak saffron in warm milk for 5 minutes and p...
2,Best Lemonade,Stephen Little,PT5M,PT30M,PT35M,This is from one of my first Good House Keepi...,[https://img.sndimg.com/food/image/upload/w_55...,Beverages,"[Low Protein, Low Cholesterol, Healthy, Summer...","[1 1⁄2, 1, None, 1 1⁄2, None, 3⁄4]","[sugar, lemons, rind of, lemon, zest of, fresh...",4.5,10.0,311.1,0.2,0.4,77.2,0.3,4.0,"[Into a 1 quart Jar with tight fitting lid, pu..."
3,Carina's Tofu-Vegetable Kebabs,Cyclopz,PT20M,PT24H,PT24H20M,This dish is best prepared a day in advance to...,[https://img.sndimg.com/food/image/upload/w_55...,Soy/Tofu,"[Beans, Vegetable, Low Cholesterol, Weeknight,...","[12, 1, 2, 1, 10, 1, 3, 2, 2, 2, 1, 2, 1⁄2, 1⁄...","[extra firm tofu, eggplant, zucchini, mushroom...",4.5,2.0,536.1,24.0,17.3,32.1,29.3,2.0,"[Drain the tofu, carefully squeezing out exces..."
4,Cabbage Soup,Duckie067,PT30M,PT20M,PT50M,Make and share this Cabbage Soup recipe from F...,[https://img.sndimg.com/food/image/upload/w_55...,Vegetable,"[Low Protein, Vegan, Low Cholesterol, Healthy,...","[46, 4, 1, 2, 1]","[plain tomato juice, cabbage, onion, carrots, ...",4.5,11.0,103.6,0.4,4.8,17.7,4.3,4.0,"[Mix everything together and bring to a boil.,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172207,Cantaloupe Margarita,Chef PotPie,PT0M,PT10M,PT10M,I've been trying all kinds of margaritas this ...,[https://img.sndimg.com/food/image/upload/w_55...,Beverages,"[< 15 Mins, Easy]","[4, 1⁄2, 1⁄4, 2, 1, 2, 2]","[cantaloupe, lime juice, sugar, coarse salt]",5.0,2.0,165.1,0.6,3.0,38.2,2.8,2.0,"[In a blender or food processor, blend the can..."
172208,Honey Paloma Cocktail,Chef PotPie,PT0M,PT20M,PT20M,Added this to the list of cocktails to try thi...,[https://img.sndimg.com/food/image/upload/w_55...,Beverages,[< 30 Mins],"[2, 1, 2, 2, 2, 1, 1⁄4, 1⁄4]","[grapefruit juice, fresh lime juice, honey syr...",5.0,1.0,285.4,0.1,0.3,75.5,0.6,1.0,"[For the honey syrup:, In a small sauce pot co..."
172209,Backyard Breakfast Egg Salad,duonyte,PT0M,PT20M,PT20M,I was gifted with eggs from the backyard coop ...,[https://img.sndimg.com/food/image/upload/w_55...,Breakfast,"[Brunch, < 30 Mins, Easy, Inexpensive, From Sc...","[6, 2, 2, 2, 4, 3, 1, 1⁄4, 1⁄4, 4]","[eggs, mayonnaise, Dijon mustard, red onions, ...",5.0,1.0,179.7,10.9,0.5,1.0,13.3,4.0,"[1. Chop the eggs., 2. Combine the mayonnaise,..."
172210,Butter Pecan Banana Muffins,jarp4,PT18M,PT10M,PT28M,Make and share this Butter Pecan Banana Muffin...,[],Breads,"[Breakfast, Brunch, < 30 Mins, Easy, From Scra...","[1, 1⁄2, 1 1⁄3, 1, 1, 1⁄2, 1, 3⁄4, 1⁄2, 1⁄4, 1...","[pecans, unsalted butter, all-purpose flour, b...",5.0,1.0,232.3,12.2,1.6,15.8,3.3,12.0,[Heat oven to 375 degrees. Coat a standard 12-...


In [306]:
# Convert arrays to lists
df_recipes2['RecipeInstructions'] = df_recipes2['RecipeInstructions'].apply(
    lambda x: list(x) if isinstance(x, (list, np.ndarray)) else []
)

### Import second dataset with units of measurement

In [307]:
df_2M = pd.read_csv('../Data/recipes_data.csv')

In [308]:
df_2M

,title,ingredients,directions,link,source,NER,site
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""bite size shredded rice biscuits"", ""vanilla""...",www.cookbooks.com
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""cream of mushroom soup"", ""beef"", ""sour cream...",www.cookbooks.com
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""pepper"", ""cream cheese"", ""gar...",www.cookbooks.com
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken gravy"", ""cream of mushroom soup"", ""c...",www.cookbooks.com
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""graham cracker crumbs"", ""powdered sugar"", ""p...",www.cookbooks.com
...,...,...,...,...,...,...,...
2231137,Sunny's Fake Crepes,"[""1/2 cup chocolate hazelnut spread (recommend...","[""Spread hazelnut spread on 1 side of each tor...",www.foodnetwork.com/recipes/sunny-anderson/sun...,Recipes1M,"[""chocolate hazelnut spread"", ""marshmallows"", ...",www.foodnetwork.com
2231138,Devil Eggs,"[""1 dozen eggs"", ""1 paprika"", ""1 salt and pepp...","[""Boil eggs on medium for 30mins."", ""Then cool...",cookpad.com/us/recipes/355411-devil-eggs,Recipes1M,"[""choice"", ""miracle whip"", ""eggs"", ""relish"", ""...",cookpad.com
2231139,Extremely Easy and Quick - Namul Daikon Salad,"[""150 grams Daikon radish"", ""1 tbsp Sesame oil...","[""Julienne the daikon and squeeze out the exce...",cookpad.com/us/recipes/153324-extremely-easy-a...,Recipes1M,"[""soy sauce"", ""radish"", ""white sesame seeds"", ...",cookpad.com
2231140,Pan-Roasted Pork Chops With Apple Fritters,"[""1 cup apple cider"", ""6 tablespoons sugar"", ""...","[""In a large bowl, mix the apple cider with 4 ...",cooking.nytimes.com/recipes/1015164,Recipes1M,"[""apple cider"", ""egg"", ""sugar"", ""freshly groun...",cooking.nytimes.com


In [311]:
# Convert strings to lists
df_2M['directions'] = df_2M['directions'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else []
)

### Merge the two datasets 

In [312]:
df_merged = pd.merge(df_recipes2, df_2M, left_on='Name', right_on='title', how='left')

In [313]:
df_merged.head()

,Name,AuthorName,CookTime,PrepTime,TotalTime,Description,Images,RecipeCategory,Keywords,RecipeIngredientQuantities,...,ProteinContent,RecipeServings,RecipeInstructions,title,ingredients,directions,link,source,NER,site
0,Low-Fat Berry Blue Frozen Dessert,Dancer,PT24H,PT45M,PT24H45M,Make and share this Low-Fat Berry Blue Frozen ...,[https://img.sndimg.com/food/image/upload/w_55...,Frozen Desserts,"[Dessert, Low Protein, Low Cholesterol, Health...","[4, 1⁄4, 1, 1]",...,3.2,4.0,"[Toss 2 cups berries with sugar., Let stand fo...",Low-Fat Berry Blue Frozen Dessert,"[""4 cups blueberries, fresh or frozen"", ""1/4 c...","[Toss 2 cups berries with sugar., Let stand fo...",www.food.com/recipe/low-fat-berry-blue-frozen-...,Gathered,"[""blueberries"", ""sugar"", ""vanilla yogurt"", ""le...",www.food.com
1,Biryani,elly9812,PT25M,PT4H,PT4H25M,Make and share this Biryani recipe from Food.com.,[https://img.sndimg.com/food/image/upload/w_55...,Chicken Breast,"[Chicken Thigh & Leg, Chicken, Poultry, Meat, ...","[1, 4, 2, 2, 8, 1⁄4, 8, 1⁄2, 1, 1, 1⁄4, 1⁄4, 1...",...,63.4,6.0,[Soak saffron in warm milk for 5 minutes and p...,Biryani,"[""1/2 lb. meat or chicken, cut in pieces"", ""1/...",[Marinate meat or chicken in thick yogurt for ...,www.cookbooks.com/Recipe-Details.aspx?id=548876,Gathered,"[""meat"", ""ginger"", ""cinnamon"", ""cloves"", ""onio...",www.cookbooks.com
2,Biryani,elly9812,PT25M,PT4H,PT4H25M,Make and share this Biryani recipe from Food.com.,[https://img.sndimg.com/food/image/upload/w_55...,Chicken Breast,"[Chicken Thigh & Leg, Chicken, Poultry, Meat, ...","[1, 4, 2, 2, 8, 1⁄4, 8, 1⁄2, 1, 1, 1⁄4, 1⁄4, 1...",...,63.4,6.0,[Soak saffron in warm milk for 5 minutes and p...,Biryani,"[""1 tablespoon saffron"", ""4 teaspoons milk, wa...",[Soak saffron in warm milk for 5 minutes and p...,www.food.com/recipe/biryani-39,Gathered,"[""tomatoes"", ""basmati rice"", ""coriander seed"",...",www.food.com
3,Biryani,elly9812,PT25M,PT4H,PT4H25M,Make and share this Biryani recipe from Food.com.,[https://img.sndimg.com/food/image/upload/w_55...,Chicken Breast,"[Chicken Thigh & Leg, Chicken, Poultry, Meat, ...","[1, 4, 2, 2, 8, 1⁄4, 8, 1⁄2, 1, 1, 1⁄4, 1⁄4, 1...",...,63.4,6.0,[Soak saffron in warm milk for 5 minutes and p...,Biryani,"[""2 cups basmati rice"", ""34 kg chicken piece"",...","[Mix tomato puree, yogurt, ginger garlic paste...",www.food.com/recipe/biryani-484928,Recipes1M,"[""oil"", ""basmati rice"", ""cumin powder"", ""tomat...",www.food.com
4,Best Lemonade,Stephen Little,PT5M,PT30M,PT35M,This is from one of my first Good House Keepi...,[https://img.sndimg.com/food/image/upload/w_55...,Beverages,"[Low Protein, Low Cholesterol, Healthy, Summer...","[1 1⁄2, 1, None, 1 1⁄2, None, 3⁄4]",...,0.3,4.0,"[Into a 1 quart Jar with tight fitting lid, pu...",Best Lemonade,"[""1 1/2 c. sugar"", ""1 Tbsp. lemon peel, finely...","[In 1-quart jar with tight fitting lid, shake ...",www.cookbooks.com/Recipe-Details.aspx?id=293155,Gathered,"[""sugar"", ""lemon juice"", ""cold water"", ""very h...",www.cookbooks.com


In [314]:
df_merged.shape[0]

3208952

#### Keep the identical recipes based on the instructions

In [315]:
filtered_df = df_merged[
    df_merged['RecipeInstructions'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None) ==
    df_merged['directions'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)
].reset_index(drop=True)

In [316]:
filtered_df

,Name,AuthorName,CookTime,PrepTime,TotalTime,Description,Images,RecipeCategory,Keywords,RecipeIngredientQuantities,...,ProteinContent,RecipeServings,RecipeInstructions,title,ingredients,directions,link,source,NER,site
0,Low-Fat Berry Blue Frozen Dessert,Dancer,PT24H,PT45M,PT24H45M,Make and share this Low-Fat Berry Blue Frozen ...,[https://img.sndimg.com/food/image/upload/w_55...,Frozen Desserts,"[Dessert, Low Protein, Low Cholesterol, Health...","[4, 1⁄4, 1, 1]",...,3.2,4.0,"[Toss 2 cups berries with sugar., Let stand fo...",Low-Fat Berry Blue Frozen Dessert,"[""4 cups blueberries, fresh or frozen"", ""1/4 c...","[Toss 2 cups berries with sugar., Let stand fo...",www.food.com/recipe/low-fat-berry-blue-frozen-...,Gathered,"[""blueberries"", ""sugar"", ""vanilla yogurt"", ""le...",www.food.com
1,Biryani,elly9812,PT25M,PT4H,PT4H25M,Make and share this Biryani recipe from Food.com.,[https://img.sndimg.com/food/image/upload/w_55...,Chicken Breast,"[Chicken Thigh & Leg, Chicken, Poultry, Meat, ...","[1, 4, 2, 2, 8, 1⁄4, 8, 1⁄2, 1, 1, 1⁄4, 1⁄4, 1...",...,63.4,6.0,[Soak saffron in warm milk for 5 minutes and p...,Biryani,"[""1 tablespoon saffron"", ""4 teaspoons milk, wa...",[Soak saffron in warm milk for 5 minutes and p...,www.food.com/recipe/biryani-39,Gathered,"[""tomatoes"", ""basmati rice"", ""coriander seed"",...",www.food.com
2,Cabbage Soup,Duckie067,PT30M,PT20M,PT50M,Make and share this Cabbage Soup recipe from F...,[https://img.sndimg.com/food/image/upload/w_55...,Vegetable,"[Low Protein, Vegan, Low Cholesterol, Healthy,...","[46, 4, 1, 2, 1]",...,4.3,4.0,"[Mix everything together and bring to a boil.,...",Cabbage Soup,"[""46 ounces plain tomato juice"", ""4 cups cabba...","[Mix everything together and bring to a boil.,...",www.food.com/recipe/cabbage-soup-42,Gathered,"[""cabbage"", ""onion"", ""celery"", ""carrots"", ""tom...",www.food.com
3,Cabbage Soup,Duckie067,PT30M,PT20M,PT50M,Make and share this Cabbage Soup recipe from F...,[https://img.sndimg.com/food/image/upload/w_55...,Vegetable,"[Low Protein, Vegan, Low Cholesterol, Healthy,...","[46, 4, 1, 2, 1]",...,4.3,4.0,"[Mix everything together and bring to a boil.,...",Cabbage Soup,"[""1 head cabbage, chopped"", ""1 onion, diced"", ...","[Mix everything together and bring to a boil.,...",www.food.com/recipe/cabbage-soup-204921,Recipes1M,"[""cabbage"", ""tomato paste"", ""pepper"", ""onion"",...",www.food.com
4,Buttermilk Pie With Gingersnap Crumb Crust,tristitia,PT50M,PT30M,PT1H20M,Make and share this Buttermilk Pie With Ginger...,[https://img.sndimg.com/food/image/upload/w_55...,Pie,"[Dessert, Healthy, Weeknight, Oven, < 4 Hours]","[3⁄4, 1, 1, 2, 3, 1⁄4, 1, 1⁄2, 1⁄2, 2]",...,4.2,8.0,"[Preheat oven to 350°F., Make pie crust, using...",Buttermilk Pie With Gingersnap Crumb Crust,"[""3/4 cup sugar"", ""1 tablespoon margarine"", ""1...","[Preheat oven to 350°F., Make pie crust, using...",www.food.com/recipe/buttermilk-pie-with-ginger...,Gathered,"[""buttermilk"", ""gingersnap crumbs"", ""egg"", ""su...",www.food.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108832,Mojo Pork Tenderloin,PanNan,PT40M,PT15M,PT55M,"Delicious and easy. When planning, please all...",[https://img.sndimg.com/food/image/upload/w_55...,Cuban,"[Caribbean, < 60 Mins]","[8, 2, 1, 1, 1, 1⁄2, 1⁄2, 1⁄2, 1⁄4, 1 1⁄2]",...,37.8,4.0,[Juice two of the oranges and all of the limes...,Mojo Pork Tenderloin,"[""8 limes, juiced"", ""2 oranges, juiced"", ""1 or...",[Juice two of the oranges and all of the limes...,www.food.com/recipe/mojo-pork-tenderloin-539091,Gathered,"[""ground cumin"", ""cilantro"", ""orange"", ""ground...",www.food.com
108833,The Soul Train,momaphet,PT0M,PT4M,PT4M,Tequila and cardamom is an unexpected combinat...,[https://img.sndimg.com/food/image/upload/w_55...,Beverages,[< 15 Mins],"[13, 1⁄2, 1⁄2, 2, 3⁄4, 1, 1⁄2, 1, None]",...,0.2,1.0,"[Simple Syrup:, Wrap cardamom in cheesecloth a...",The Soul Train,"[""Cardamom Simple Syrup"", ""13 cardamom pods, l.

In [317]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108837 entries, 0 to 108836
Data columns (total 27 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Name                        108837 non-null  object 
 1   AuthorName                  108837 non-null  object 
 2   CookTime                    108837 non-null  object 
 3   PrepTime                    108837 non-null  object 
 4   TotalTime                   108837 non-null  object 
 5   Description                 108837 non-null  object 
 6   Images                      108837 non-null  object 
 7   RecipeCategory              108837 non-null  object 
 8   Keywords                    108837 non-null  object 
 9   RecipeIngredientQuantities  108837 non-null  object 
 10  RecipeIngredientParts       108837 non-null  object 
 11  AggregatedRating            108837 non-null  float64
 12  ReviewCount                 108837 non-null  float64
 13  Calories      

#### Drop duplicate columns

In [328]:
columns_to_drop = ['Name', 'RecipeIngredientQuantities', 'RecipeIngredientParts', 'RecipeInstructions']
df = filtered_df.drop(columns=columns_to_drop)

In [329]:
df

,AuthorName,CookTime,PrepTime,TotalTime,Description,Images,RecipeCategory,Keywords,AggregatedRating,ReviewCount,...,SugarContent,ProteinContent,RecipeServings,title,ingredients,directions,link,source,NER,site
0,Dancer,PT24H,PT45M,PT24H45M,Make and share this Low-Fat Berry Blue Frozen ...,[https://img.sndimg.com/food/image/upload/w_55...,Frozen Desserts,"[Dessert, Low Protein, Low Cholesterol, Health...",4.5,4.0,...,30.2,3.2,4.0,Low-Fat Berry Blue Frozen Dessert,"[""4 cups blueberries, fresh or frozen"", ""1/4 c...","[Toss 2 cups berries with sugar., Let stand fo...",www.food.com/recipe/low-fat-berry-blue-frozen-...,Gathered,"[""blueberries"", ""sugar"", ""vanilla yogurt"", ""le...",www.food.com
1,elly9812,PT25M,PT4H,PT4H25M,Make and share this Biryani recipe from Food.com.,[https://img.sndimg.com/food/image/upload/w_55...,Chicken Breast,"[Chicken Thigh & Leg, Chicken, Poultry, Meat, ...",3.0,1.0,...,20.4,63.4,6.0,Biryani,"[""1 tablespoon saffron"", ""4 teaspoons milk, wa...",[Soak saffron in warm milk for 5 minutes and p...,www.food.com/recipe/biryani-39,Gathered,"[""tomatoes"", ""basmati rice"", ""coriander seed"",...",www.food.com
2,Duckie067,PT30M,PT20M,PT50M,Make and share this Cabbage Soup recipe from F...,[https://img.sndimg.com/food/image/upload/w_55...,Vegetable,"[Low Protein, Vegan, Low Cholesterol, Healthy,...",4.5,11.0,...,17.7,4.3,4.0,Cabbage Soup,"[""46 ounces plain tomato juice"", ""4 cups cabba...","[Mix everything together and bring to a boil.,...",www.food.com/recipe/cabbage-soup-42,Gathered,"[""cabbage"", ""onion"", ""celery"", ""carrots"", ""tom...",www.food.com
3,Duckie067,PT30M,PT20M,PT50M,Make and share this Cabbage Soup recipe from F...,[https://img.sndimg.com/food/image/upload/w_55...,Vegetable,"[Low Protein, Vegan, Low Cholesterol, Healthy,...",4.5,11.0,...,17.7,4.3,4.0,Cabbage Soup,"[""1 head cabbage, chopped"", ""1 onion, diced"", ...","[Mix everything together and bring to a boil.,...",www.food.com/recipe/cabbage-soup-204921,Recipes1M,"[""cabbage"", ""tomato paste"", ""pepper"", ""onion"",...",www.food.com
4,tristitia,PT50M,PT30M,PT1H20M,Make and share this Buttermilk Pie With Ginger...,[https://img.sndimg.com/food/image/upload/w_55...,Pie,"[Dessert, Healthy, Weeknight, Oven, < 4 Hours]",4.0,3.0,...,24.7,4.2,8.0,Buttermilk Pie With Gingersnap Crumb Crust,"[""3/4 cup sugar"", ""1 tablespoon margarine"", ""1...","[Preheat oven to 350°F., Make pie crust, using...",www.food.com/recipe/buttermilk-pie-with-ginger...,Gathered,"[""buttermilk"", ""gingersnap crumbs"", ""egg"", ""su...",www.food.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108832,PanNan,PT40M,PT15M,PT55M,"Delicious and easy. When planning, please all...",[https://img.sndimg.com/food/image/upload/w_55...,Cuban,"[Caribbean, < 60 Mins]",5.0,1.0,...,11.6,37.8,4.0,Mojo Pork Tenderloin,"[""8 limes, juiced"", ""2 oranges, juiced"", ""1 or...",[Juice two of the oranges and all of the limes...,www.food.com/recipe/mojo-pork-tenderloin-539091,Gathered,"[""ground cumin"", ""cilantro"", ""orange"", ""ground...",www.food.com
108833,momaphet,PT0M,PT4M,PT4M,Tequila and cardamom is an unexpected combinat...,[https://img.sndimg.com/food/image/upload/w_55...,Beverages,[< 15 Mins],5.0,3.0,...,102.8,0.2,1.0,The Soul Train,"[""Cardamom Simple Syrup"", ""13 cardamom pods, l...","[Simple Syrup:, Wrap cardamom in cheesecloth a...",www.food.com/recipe/the-soul-train-539095,Gathered,"[""sugar"", ""cardamom simple syrup"", ""freshly sq...",www.food.com
108834,momaphet,PT0M,PT3M,PT3M,This cocktail was developed for Mary Pickford ...,[https://img.sndimg.com/food/image/upload/w_55...,Beverages,"[Low Protein, Low Cholesterol, Healthy, < 15 M...",5.0,3.0,...,16.9,0.2,1.0,Mary Pickford Cocktail,"[""2 ounces light rum"", ""2 ounces pineapple jui...",[Pour all ingredients except the cherry into a...,www.food.com/recipe/mary-pickford-cocktail-539102,Gathered,"[""light rum"", ""grenadine"", ""pineapple juice"", ...",www.food.com
1088

## Cleaning

### Columns CookTime, PrepTime and TotalTime

In [320]:
def duration_to_minutes(duration: str) -> float:
    """
    Function to convert ISO 8601 durations to total minutes

    Args:
        - duration (str): duration in ISO 8601 format (ex: 'PT1H30M')
    """
    if pd.isna(duration): 
        return np.nan

    hours = re.search(r'(\d+)H', duration)
    minutes = re.search(r'(\d+)M', duration)
    
    total_minutes = 0
    if hours:
        total_minutes += int(hours.group(1)) * 60
    if minutes:
        total_minutes += int(minutes.group(1))
    
    return total_minutes

In [ ]:
def duration_to_readable_format(duration: str) -> str:
    """
    Function to convert ISO 8601 durations to a more readable format
    
    Args:
        - duration (str): duration in ISO 8601 format (ex: 'PT1H30M')

    Example output: "1 h 30 min"
    """
    if pd.isna(duration): 
        return np.nan

    hours = re.search(r'(\d+)H', duration)
    minutes = re.search(r'(\d+)M', duration)
    
    result = []
    if hours:
        result.append(f"{int(hours.group(1))} h")
    if minutes:
        result.append(f"{int(minutes.group(1))} min")
    
    return ' '.join(result)

In [326]:
duration_to_readable_format('PT1H30M')

'1 h 30 min'

In [330]:
columns = ['CookTime', 'PrepTime', 'TotalTime']
for col in columns:
    df[f'{col}_min'] = df[col].apply(duration_to_minutes)

### Column Images : keep the first link only

In [332]:
def extract_first_element(x):
    if isinstance(x, np.ndarray): 
        return x[0] if x.size > 0 else None  
    elif pd.notna(x):  
        return x  
    else:
        return None

In [333]:
df['Images'] = df['Images'].apply(extract_first_element)

In [334]:
df

,AuthorName,CookTime,PrepTime,TotalTime,Description,Images,RecipeCategory,Keywords,AggregatedRating,ReviewCount,...,title,ingredients,directions,link,source,NER,site,CookTime_min,PrepTime_min,TotalTime_min
0,Dancer,PT24H,PT45M,PT24H45M,Make and share this Low-Fat Berry Blue Frozen ...,https://img.sndimg.com/food/image/upload/w_555...,Frozen Desserts,"[Dessert, Low Protein, Low Cholesterol, Health...",4.5,4.0,...,Low-Fat Berry Blue Frozen Dessert,"[""4 cups blueberries, fresh or frozen"", ""1/4 c...","[Toss 2 cups berries with sugar., Let stand fo...",www.food.com/recipe/low-fat-berry-blue-frozen-...,Gathered,"[""blueberries"", ""sugar"", ""vanilla yogurt"", ""le...",www.food.com,1440,45,1485
1,elly9812,PT25M,PT4H,PT4H25M,Make and share this Biryani recipe from Food.com.,https://img.sndimg.com/food/image/upload/w_555...,Chicken Breast,"[Chicken Thigh & Leg, Chicken, Poultry, Meat, ...",3.0,1.0,...,Biryani,"[""1 tablespoon saffron"", ""4 teaspoons milk, wa...",[Soak saffron in warm milk for 5 minutes and p...,www.food.com/recipe/biryani-39,Gathered,"[""tomatoes"", ""basmati rice"", ""coriander seed"",...",www.food.com,25,240,265
2,Duckie067,PT30M,PT20M,PT50M,Make and share this Cabbage Soup recipe from F...,https://img.sndimg.com/food/image/upload/w_555...,Vegetable,"[Low Protein, Vegan, Low Cholesterol, Healthy,...",4.5,11.0,...,Cabbage Soup,"[""46 ounces plain tomato juice"", ""4 cups cabba...","[Mix everything together and bring to a boil.,...",www.food.com/recipe/cabbage-soup-42,Gathered,"[""cabbage"", ""onion"", ""celery"", ""carrots"", ""tom...",www.food.com,30,20,50
3,Duckie067,PT30M,PT20M,PT50M,Make and share this Cabbage Soup recipe from F...,https://img.sndimg.com/food/image/upload/w_555...,Vegetable,"[Low Protein, Vegan, Low Cholesterol, Healthy,...",4.5,11.0,...,Cabbage Soup,"[""1 head cabbage, chopped"", ""1 onion, diced"", ...","[Mix everything together and bring to a boil.,...",www.food.com/recipe/cabbage-soup-204921,Recipes1M,"[""cabbage"", ""tomato paste"", ""pepper"", ""onion"",...",www.food.com,30,20,50
4,tristitia,PT50M,PT30M,PT1H20M,Make and share this Buttermilk Pie With Ginger...,https://img.sndimg.com/food/image/upload/w_555...,Pie,"[Dessert, Healthy, Weeknight, Oven, < 4 Hours]",4.0,3.0,...,Buttermilk Pie With Gingersnap Crumb Crust,"[""3/4 cup sugar"", ""1 tablespoon margarine"", ""1...","[Preheat oven to 350°F., Make pie crust, using...",www.food.com/recipe/buttermilk-pie-with-ginger...,Gathered,"[""buttermilk"", ""gingersnap crumbs"", ""egg"", ""su...",www.food.com,50,30,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108832,PanNan,PT40M,PT15M,PT55M,"Delicious and easy. When planning, please all...",https://img.sndimg.com/food/image/upload/w_555...,Cuban,"[Caribbean, < 60 Mins]",5.0,1.0,...,Mojo Pork Tenderloin,"[""8 limes, juiced"", ""2 oranges, juiced"", ""1 or...",[Juice two of the oranges and all of the limes...,www.food.com/recipe/mojo-pork-tenderloin-539091,Gathered,"[""ground cumin"", ""cilantro"", ""orange"", ""ground...",www.food.com,40,15,55
108833,momaphet,PT0M,PT4M,PT4M,Tequila and cardamom is an unexpected combinat...,https://img.sndimg.com/food/image/upload/w_555...,Beverages,[< 15 Mins],5.0,3.0,...,The Soul Train,"[""Cardamom Simple Syrup"", ""13 cardamom pods, l...","[Simple Syrup:, Wrap cardamom in cheesecloth a...",www.food.com/recipe/the-soul-train-539095,Gathered,"[""sugar"", ""cardamom simple syrup"", ""freshly sq...",www.food.com,0,4,4
108834,momaphet,PT0M,PT3M,PT3M,This cocktail was developed for Mary Pickford ...,https://img.sndimg.com/food/image/upload/w_555...,Beverages,"[Low Protein, Low Cholesterol, Healthy, < 15 M...",5.0,3.0,...,Mary Pickford Cocktail,"[""2 ounces light rum"", ""2 ounces pineapple jui...",[Pour all ingredients except the cherry into a...,www.food.com/recipe/mary-pickford-cocktail-539102,Gathered,"[""light rum"", ""grenadine"", ""pineapple juice"", ...",www.food.com,0,3,3
108835,Chef PotPie,PT30M,PT10M,PT40M,"A tr

In [336]:
df['Images']

0         https://img.sndimg.com/food/image/upload/w_555...
1         https://img.sndimg.com/food/image/upload/w_555...
2         https://img.sndimg.com/food/image/upload/w_555...
3         https://img.sndimg.com/food/image/upload/w_555...
4         https://img.sndimg.com/food/image/upload/w_555...
                                ...                        
108832    https://img.sndimg.com/food/image/upload/w_555...
108833    https://img.sndimg.com/food/image/upload/w_555...
108834    https://img.sndimg.com/food/image/upload/w_555...
108835    https://img.sndimg.com/food/image/upload/w_555...
108836    https://img.sndimg.com/food/image/upload/w_555...
Name: Images, Length: 108837, dtype: object

In [337]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108837 entries, 0 to 108836
Data columns (total 26 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   AuthorName        108837 non-null  object 
 1   CookTime          108837 non-null  object 
 2   PrepTime          108837 non-null  object 
 3   TotalTime         108837 non-null  object 
 4   Description       108837 non-null  object 
 5   Images            55560 non-null   object 
 6   RecipeCategory    108837 non-null  object 
 7   Keywords          108837 non-null  object 
 8   AggregatedRating  108837 non-null  float64
 9   ReviewCount       108837 non-null  float64
 10  Calories          108837 non-null  float64
 11  FatContent        108837 non-null  float64
 12  FiberContent      108837 non-null  float64
 13  SugarContent      108837 non-null  float64
 14  ProteinContent    108837 non-null  float64
 15  RecipeServings    108837 non-null  float64
 16  title             10

### Columns RecipeCategory and Keywords

In [338]:
df['RecipeCategory'].value_counts()[:50]

Dessert                11720
Lunch/Snacks            7901
One Dish Meal           6613
Vegetable               6597
Beverages               5311
Breakfast               4960
Chicken                 3444
Chicken Breast          3141
Pork                    2786
Meat                    2753
Breads                  2734
Potato                  2622
Quick Breads            2549
Pie                     1832
Sauces                  1712
Cheese                  1673
Bar Cookie              1532
< 30 Mins               1500
< 60 Mins               1419
Smoothies               1280
Stew                    1177
< 15 Mins               1080
Drop Cookies            1043
Low Protein             1030
Beans                   1019
Yeast Breads             945
Frozen Desserts          872
Salad Dressings          853
Poultry                  852
Candy                    817
Steak                    803
Cheesecake               795
Spreads                  750
Onions                   669
Curries       

In [339]:
df['RecipeCategory'].nunique()

267

#### Group categories

- Dessert 
- Main course
- Breakfast
- Beverages

In [340]:
df['Keywords']

0         [Dessert, Low Protein, Low Cholesterol, Health...
1         [Chicken Thigh & Leg, Chicken, Poultry, Meat, ...
2         [Low Protein, Vegan, Low Cholesterol, Healthy,...
3         [Low Protein, Vegan, Low Cholesterol, Healthy,...
4            [Dessert, Healthy, Weeknight, Oven, < 4 Hours]
                                ...                        
108832                               [Caribbean, < 60 Mins]
108833                                          [< 15 Mins]
108834    [Low Protein, Low Cholesterol, Healthy, < 15 M...
108835                         [Dutch, European, < 60 Mins]
108836    [European, Low Protein, Low Cholesterol, Healt...
Name: Keywords, Length: 108837, dtype: object

In [341]:
from collections import Counter
all_keywords = [keyword for keywords in df['Keywords'] for keyword in keywords]
keyword_counts = Counter(all_keywords)
keyword_counts_df = pd.DataFrame(keyword_counts.items(), columns=['Keyword', 'Count'])
keyword_counts_df = keyword_counts_df.sort_values(by='Count', ascending=False)


In [342]:
keyword_counts_df['Keyword'].unique()

array(['Easy', '< 60 Mins', '< 30 Mins', 'Meat', '< 15 Mins', '< 4 Hours',
       'Healthy', 'Vegetable', 'Low Cholesterol', 'Oven', 'Beginner Cook',
       'Fruit', 'Low Protein', 'Inexpensive', 'Kid Friendly',
       'For Large Groups', 'Poultry', 'Stove Top', 'European',
       'Weeknight', 'Dessert', 'Brunch', 'Chicken', 'Asian', 'High In...',
       'Free Of...', 'Summer', 'Potluck', 'Cheese', 'Sweet',
       'Cookie & Brownie', 'Vegan', 'Savory', 'Very Low Carbs', 'Breads',
       'Small Appliance', 'High Protein', 'Lunch/Snacks', 'Spicy',
       'Winter', 'No Cook', 'Christmas', 'Mexican', 'Beans', 'Nuts',
       'Tropical Fruits', 'Berries', 'Pork', None, 'Refrigerator',
       'Beverages', 'Apple', 'Canadian', 'Grains', 'Egg Free', 'Rice',
       'Spring', 'Kosher', 'Breakfast', 'Potato', 'Thanksgiving',
       'Citrus', 'Toddler Friendly', 'Lactose Free', 'Southwestern U.S.',
       'Onions', 'Indian', 'Greens', 'Chicken Breast', 'African',
       'Strawberry', 'Australian', 

In [350]:
trigger_patterns = {
    'Main Course': r'lunch|meal|meat|chicken|beef|pork|steak|turkey|duck|fish|salmon|lamb|crab|lobster|tuna|vegetable|potato|rice|pasta|penne|spaghetti|bean|lentil|onion|soup|stew|dressing',
    'Breakfast': r'breakfast',
    'Dessert': r'dessert|cake|cookie|brownie|sweet|candy|sugar|banana',
    'Beverages': r'beverage|cocktail|smoothie|lemonade|coffee',
}

In [351]:
def assign_category(row):
    keywords = row['Keywords']
    title = row['title'] 
    category = row['RecipeCategory']
    ingredients = row['NER']

    # Check the Keywords first
    if keywords is not None and isinstance(keywords, (list, np.ndarray)):
        keywords_list = [str(keyword) for keyword in keywords if keyword is not None and not pd.isna(keyword)]
        if keywords_list:
            keywords_str = ' '.join(keywords_list).lower()
            for cat, pattern in trigger_patterns.items():
                if re.search(pattern, keywords_str):
                    return cat
    
    # Check the title
    if title is not None and isinstance(title, str):
        title_lower = title.lower()
        for cat, pattern in trigger_patterns.items():
            if re.search(pattern, title_lower):
                return cat
    
    # Check the RecipeCategory
    if category is not None and isinstance(category, str):
        category_lower = category.lower()
        for cat, pattern in trigger_patterns.items():
            if re.search(pattern, category_lower):
                return cat
    
    # # Check the ingredients
    # if ingredients is not None and isinstance(ingredients, str):
    #     ingredients_list = ast.literal_eval(ingredients)
    #     if isinstance(ingredients_list, list):
    #         ingredients_str = ' '.join(map(str, ingredients_list)).lower()
    #         for cat, pattern in trigger_patterns.items():
    #             if re.search(pattern, ingredients_str):
    #                 return cat
    
    return row['RecipeCategory']

In [352]:
df['RecipeType'] = df.apply(assign_category, axis=1)

In [353]:
df['RecipeType'].nunique()

167

In [354]:
df['RecipeType'].value_counts()[:50]

Main Course         64486
Dessert             21442
Beverages            5908
Breakfast            3377
Breads               1297
Quick Breads          995
< 30 Mins             931
< 15 Mins             758
Sauces                746
< 60 Mins             739
Yeast Breads          682
Low Protein           650
Cheese                635
European              410
Spreads               327
Very Low Carbs        303
Low Cholesterol       291
Fruit                 261
< 4 Hours             251
Mexican               240
Asian                 234
Grains                189
Healthy               171
Apple                 151
Kid Friendly          135
Scones                124
Pineapple             123
Tilapia               111
Weeknight             100
Chowders               96
High In...             85
High Protein           78
Greek                  77
Halibut                75
For Large Groups       69
Brunch                 68
Cajun                  68
Vegan                  66
Potluck     

In [355]:
df[['title', 'RecipeType', 'RecipeCategory', 'Keywords']][df['RecipeCategory']=='Breads'][:50]

,title,RecipeType,RecipeCategory,Keywords
29,Carrot Ginger Biscuits,Breads,Breads,"[Low Cholesterol, Healthy, < 60 Mins, Oven]"
278,Basil Parmesan Biscuits,Breads,Breads,"[Low Cholesterol, Healthy, High In..., < 30 Mi..."
340,Amish Biscuits,Breads,Breads,"[< 15 Mins, For Large Groups, Oven]"
400,Texas Spoon Bread,Breads,Breads,"[Tex Mex, Southwestern U.S., < 60 Mins, Oven]"
421,Old Fashioned Crumb Cake,Dessert,Breads,"[< 60 Mins, Oven, Easy]"
460,Herbed Pizza Crust,Breads,Breads,"[Low Cholesterol, Healthy, Bread Machine, < 30..."
551,Cinnamon Peach Coffee Cake,Dessert,Breads,"[Dessert, Fruit, Brunch, Oven, < 4 Hours]"
605,Focaccia with Rosemary & Olives,Breads,Breads,"[European, Potluck, < 60 Mins, Oven]"
606,Focaccia,Breads,Breads,"[European, Low Cholesterol, Healthy, Oven, < 4..."
648,Aunt Mary's Biscuits,Breads,Breads,"[< 30 Mins, Oven]"
